# Mount the Gdrive


In [1]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [2]:
!ls "/content/gdrive/My Drive/eva4p2/drone-dataset1.zip"

'/content/gdrive/My Drive/eva4p2/drone-dataset1.zip'


In [3]:
cp -r "/content/gdrive/My Drive/eva4p2/drone-dataset1.zip" "drone-dataset.zip"

In [4]:
!unzip drone-dataset.zip

Streaming output truncated to the last 5000 lines.
  inflating: drone-dataset/train/3/Image_67.jpg  
  inflating: drone-dataset/train/3/sandhu-eva4p2-imgg0174.jpg  
  inflating: drone-dataset/train/3/wing_Drone_0_B _95_.jpg  
  inflating: drone-dataset/train/3/capture1.jpg  
  inflating: drone-dataset/train/3/dsc07909.jpg  
  inflating: drone-dataset/train/3/image221.jpg  
  inflating: drone-dataset/train/3/217095_web.jpg  
  inflating: drone-dataset/train/3/5e31b98be0.jpg  
  inflating: drone-dataset/train/3/Image_316.jpg  
  inflating: drone-dataset/train/3/Image_417.jpg  
  inflating: drone-dataset/train/3/Image_692.jpg  
  inflating: drone-dataset/train/3/Tellus4D-2.jpg  
  inflating: drone-dataset/train/3/Image_144.jpg  
  inflating: drone-dataset/train/3/Image_293.jpg  
  inflating: drone-dataset/train/3/SkyWalker-X8-Heliguy-crop(1).jpg  
  inflating: drone-dataset/train/3/685080fc37bf3b1a28f92578413146d5.jpg  
  inflating: drone-dataset/train/3/dt26-open-payload-package.jpg  
  

#Train/Test loader

In [5]:
import torchvision
import torch
import torchvision.transforms as transforms

def load_dataset():
    train_data_path = '/content/drone-dataset/train'
    test_data_path = '/content/drone-dataset/test'
    train_dataset = torchvision.datasets.ImageFolder(
        root=train_data_path,
        transform = transforms.Compose([
        transforms.Resize((224,224)),
        transforms.Grayscale(num_output_channels=3),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
    )
    train_loader = torch.utils.data.DataLoader(
        train_dataset,
        batch_size=16,
        num_workers=0,
        shuffle=True
    )
    
    test_dataset = torchvision.datasets.ImageFolder(
        root=test_data_path,
        transform = transforms.Compose([
        transforms.Resize((224,224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
    )
    test_loader = torch.utils.data.DataLoader(
        test_dataset,
        batch_size=16,
        num_workers=0,
        shuffle=True
    )
    return train_loader, test_loader

train_loader, test_loader = load_dataset()

#Model Used

In [7]:
import torch
import torch.nn.functional as F
import torch.nn as nn

class DroneMobilenetV2(torch.nn.Module):
    def __init__(self):
        super().__init__()
        # Use ROIpoolng to resize image to 224*224
        #self.ROIpool = nn.AdaptiveMaxPool2d((224,224))
        # Use a pretrained MobilenetV2 model
        self.network = torch.hub.load('pytorch/vision:v0.6.0', 'mobilenet_v2', pretrained=True)
        # Replace last layer
        num_ftrs = self.network.classifier[1].in_features
        #Add fully connected layer
        self.network.classifier[1] = nn.Linear(num_ftrs, 4) # output 4 classifications
    
    def forward(self, xb):
        #resized = self.ROIpool(xb)
        out     = self.network(xb)
        return F.log_softmax(out)
#         return torch.sigmoid(self.fc(self.relu(self.network(xb))))

    
    def freeze(self):
        # To freeze the residual layers
        for param in self.network.parameters():
            param.require_grad = False
        for param in self.network.classifier[1].parameters():
            param.require_grad = True
    
    def unfreeze(self):
        # Unfreeze all layers
        for param in self.network.parameters():
            param.require_grad = True

#Show Summary

In [8]:
import torch
import torch.optim as optim
from torchsummary import summary

model = DroneMobilenetV2().cuda()
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

summary(model, input_size=(3,224,224))


Downloading: "https://github.com/pytorch/vision/archive/v0.6.0.zip" to /root/.cache/torch/hub/v0.6.0.zip
Downloading: "https://download.pytorch.org/models/mobilenet_v2-b0353104.pth" to /root/.cache/torch/hub/checkpoints/mobilenet_v2-b0353104.pth



----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 32, 112, 112]             864
       BatchNorm2d-2         [-1, 32, 112, 112]              64
             ReLU6-3         [-1, 32, 112, 112]               0
            Conv2d-4         [-1, 32, 112, 112]             288
       BatchNorm2d-5         [-1, 32, 112, 112]              64
             ReLU6-6         [-1, 32, 112, 112]               0
            Conv2d-7         [-1, 16, 112, 112]             512
       BatchNorm2d-8         [-1, 16, 112, 112]              32
  InvertedResidual-9         [-1, 16, 112, 112]               0
           Conv2d-10         [-1, 96, 112, 112]           1,536
      BatchNorm2d-11         [-1, 96, 112, 112]             192
            ReLU6-12         [-1, 96, 112, 112]               0
           Conv2d-13           [-1, 96, 56, 56]             864
      BatchNorm2d-14           [-1, 96

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:20: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


#Freeze the Model

In [9]:

model.freeze()

# Train, Test utility functions

In [10]:
import torch.nn as nn
from tqdm import tqdm
import torch.optim as optim

train_losses = []
train_acc = []

def train(model, device, trainloader, epoch):
    running_loss = 0.00
    correct = 0.0
    processed = 0
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
    pbar = tqdm(trainloader)
    for i, data in enumerate(pbar):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        train_losses.append(loss)
        loss.backward()
        optimizer.step()
        output = outputs.argmax(dim=1, keepdim=True)
        correct = correct + output.eq(labels.view_as(output)).sum().item()
        processed = processed + len(inputs)
        pbar.set_description(desc= f'Loss={loss.item()} Accuracy={100*correct/processed:.2f}')
        train_acc.append(100*correct/processed)

import torch
import torch.nn.functional as F

test_losses = []
test_acc = []

def test(model, device, testloader):
    correct = 0
    total = 0
    test_loss = 0
    with torch.no_grad():
        for data in testloader:
            images, labels = data
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            test_loss += F.nll_loss(outputs, labels, reduction='sum').item()
            #criterion = nn.CrossEntropyLoss()
            #test_loss += criterion(outputs, labels, reduction='sum').item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
    test_losses.append(test_loss)
    print('\nTest Set: Average loss: {}, Accuracy: {}/{} ({:.2f}%)\n'.format(test_loss, correct, len(testloader.dataset),
                                                                        100. * correct / len(testloader.dataset)))


# Train the data

In [11]:
for epoch in range(5):
    train(model, device, train_loader, epoch)
    test(model, device, test_loader)
    torch.save(model, 'drone_mobilenet_v2_3.pt')
    print("*********** Epoch {} ended *************".format(epoch))

Loss=0.12595342099666595 Accuracy=93.75:   0%|          | 1/707 [00:00<01:29,  7.89it/s]


Test Set: Average loss: 1746.4135016202927, Accuracy: 4240/4826 (87.86%)

*********** Epoch 0 ended *************


  0%|          | 0/707 [00:00<?, ?it/s]


Test Set: Average loss: 1606.6312779337168, Accuracy: 4334/4826 (89.81%)

*********** Epoch 1 ended *************


  0%|          | 0/707 [00:00<?, ?it/s]


Test Set: Average loss: 1508.889540374279, Accuracy: 4318/4826 (89.47%)

*********** Epoch 2 ended *************


  0%|          | 0/707 [00:00<?, ?it/s]


Test Set: Average loss: 1402.6506829410791, Accuracy: 4356/4826 (90.26%)

*********** Epoch 3 ended *************


Loss=7.402228832244873 Accuracy=96.52: 100%|██████████| 707/707 [04:14<00:00,  2.77it/s]



Test Set: Average loss: 1462.7111346125603, Accuracy: 4418/4826 (91.55%)

*********** Epoch 4 ended *************


###The below content is a copy paste about mobilenet information..
### This notebook is optionally accelerated with a GPU runtime.
### If you would like to use this acceleration, please select the menu option "Runtime" -> "Change runtime type", select "Hardware Accelerator" -> "GPU" and click "SAVE"

----------------------------------------------------------------------

# MobileNet v2

*Author: Pytorch Team*

**Efficient networks optimized for speed and memory, with residual blocks**

_ | _
- | -
![alt](https://pytorch.org/assets/images/mobilenet_v2_1.png) | ![alt](https://pytorch.org/assets/images/mobilenet_v2_2.png)

In [ ]:
import torch
#define the model
model = torch.hub.load('pytorch/vision:v0.6.0', 'mobilenet_v2', pretrained=True)
model.eval()

In [ ]:
#trace model with a dummy input
traced_model = torch.jit.trace(model, torch.randn(1,3,224,224))
traced_model.save('mobilenetV2.pt')

All pre-trained models expect input images normalized in the same way,
i.e. mini-batches of 3-channel RGB images of shape `(3 x H x W)`, where `H` and `W` are expected to be at least `224`.
The images have to be loaded in to a range of `[0, 1]` and then normalized using `mean = [0.485, 0.456, 0.406]`
and `std = [0.229, 0.224, 0.225]`.

Here's a sample execution.

In [ ]:
# Download an example image from the pytorch website
import urllib
url, filename = ("https://github.com/pytorch/hub/raw/master/dog.jpg", "dog.jpg")
try: urllib.URLopener().retrieve(url, filename)
except: urllib.request.urlretrieve(url, filename)

In [ ]:
# sample execution (requires torchvision)
from PIL import Image
from torchvision import transforms
input_image = Image.open(filename)
preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])
input_tensor = preprocess(input_image)
input_batch = input_tensor.unsqueeze(0) # create a mini-batch as expected by the model

# move the input and model to GPU for speed if available
if torch.cuda.is_available():
    input_batch = input_batch.to('cuda')
    model.to('cuda')

with torch.no_grad():
    output = model(input_batch)
# Tensor of shape 1000, with confidence scores over Imagenet's 1000 classes
print(output[0])
# The output has unnormalized scores. To get probabilities, you can run a softmax on it.
print(torch.nn.functional.softmax(output[0], dim=0))


### Model Description

The MobileNet v2 architecture is based on an inverted residual structure where the input and output of the residual block are thin bottleneck layers opposite to traditional residual models which use expanded representations in the input. MobileNet v2 uses lightweight depthwise convolutions to filter features in the intermediate expansion layer. Additionally, non-linearities in the narrow layers were removed in order to maintain representational power.

| Model structure | Top-1 error | Top-5 error |
| --------------- | ----------- | ----------- |
|  mobilenet_v2       | 28.12       | 9.71       |


### References

 - [MobileNetV2: Inverted Residuals and Linear Bottlenecks](https://arxiv.org/abs/1801.04381)